# Circuit Distribution and Basic Usage

`pytket-dqc` is a python package for the distribution of quantum circuits between interconnected quantum computers, which we call modules (or servers). Here we gather some basic usage of `pytket-dqc`, and introduce some basic concepts from distributed quantum computing (DQC). For more extensive discussion on `pytket-dqc` please see the documentation [here](https://cqcl.github.io/pytket-dqc/).

The main data structures we will discuss include: `NISQNetwork`, the network on which a circuit will be distributed; `Circuit`, describing the circuit to be distributed; `Distribution`, which describes the distribution of a circuit onto a network; and `Distributor`, the parent class for a collection of techniques for generating a `Distribution`. We will also introduce the notions of starting and ending processes, link qubits, detached gates and Steiner trees, as discussed in greater detail in the corresponding paper [Distributing circuits over heterogeneous, modular quantum computing network architectures](link).

## Networks

Networks of quantum servers are specified by two properties:
- The server coupling, detailing which servers are connected to which others. In `pytket-dqc` this is specified by a list of pairs of integers, where each pair signifies that there is a connection between those two servers. 
- The qubits each server contains. This is specified by a dictionary from the server to a list of qubits it contains. 

An example is given below, where blue lines indicate connections between modules, and red lines indicate connections between qubits within modules. The vertex labels are the indexes of the qubits.

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2], [0,3]], {0:[0,1,2,3], 1:[4,5,6], 2:[7,8,9,10,11], 3:[12,13]})
f = network.draw_nisq_network()

## Circuits

Note that the accepted gate set for `pytket-dqc` is CU1, Rz, and H. While restrictive, this is a universal gate set, and `pytket-dqc` includes some handy utilities for rebasing your circuit if it is not in the correct gate set initially.

In [ ]:
from pytket_dqc.utils import DQCPass
from pytket.circuit.display import render_circuit_jupyter
from pytket import Circuit

circ = Circuit(4).CY(0,1).CZ(1,2).H(1).X(3).CX(1,0).CZ(1,3)
render_circuit_jupyter(circ)

DQCPass().apply(circ)
render_circuit_jupyter(circ)

## Distributors and Distributions

`pytket-dqc` includes several instances of the `Distributor` class, each of which convert a `Circuit` and a `NISQNetwork` into a `Distribution`. A `Distribution` describes how and where gates should be acted in the network, and where qubits of the original circuit should be assigned. To demonstrate this let us first introduce a small network and circuit.

In [ ]:
from pytket import OpType

circ = Circuit(2)
circ.add_gate(OpType.CU1, 1.0, [0, 1]).H(0).Rz(0.3,0).H(0).add_gate(OpType.CU1, 1.0, [0, 1])
render_circuit_jupyter(circ)

network = NISQNetwork([[0,1], [0,2]], {0:[0], 1:[1], 2:[2]})
f = network.draw_nisq_network()

One example of an available `Distributor` is `PartitioningAnnealing`, which uses simulated annealing to distribute a circuit to a network. Other distributors are discussed further in the notebook `distributor_comparison`. The resulting `Distribution` includes several useful methods, including the generation of a `pytket` `Circuit` which includes the operations required to implement it in a distributed way, as is demonstrated below.

In [ ]:
from pytket_dqc.distributors import PartitioningAnnealing

distribution = PartitioningAnnealing().distribute(circ, network, seed=1)

distributed_circ = distribution.to_pytket_circuit()
render_circuit_jupyter(distributed_circ)

print('Distributed Circuit Commands:', *distributed_circ.get_commands(), sep='\n')

Here we see that the qubits in the original circuit have been assigned to particular modules in the network, that two instances of `CustomGate` have been added to the circuit, and that an additional ancilla has been added to the circuit. The first and second `CustomGate` are a starting processes and ending process respectively, as seen in the corresponding list of commands. These together implement the single EJPP process required to distribute the two 2-qubit gates in the circuit. Please see [Optimal local implementation of non-local quantum gates](https://arxiv.org/abs/quant-ph/0005101) for further details on the EJPP protocol itself.

## Remote Gates 

As well as facilitating the automatic generation of a `Distribution`, `pytket-dqc` allows for a custom `Distribution` to be defined. This requires, in addition to a `NISQNetwork` as discussed above, a `HypergrapCircuit`, which manages some additional information about a `pytket` `Circuit`, and a `Placement`, which describes where gates and qubits are assigned. The placement below assigns the two qubits in the circuit to the modules at the ends of the line network, and assigns the two gates to the central module.

In [ ]:
from pytket_dqc.placement import Placement
from pytket_dqc import Distribution
from pytket_dqc.circuits import HypergraphCircuit

hyp_circ = HypergraphCircuit(circ)
placement = Placement({0:1, 1:2, 2:0, 3:0})
distribution = Distribution(hyp_circ, placement, network)
assert distribution.is_valid()

circ_with_dist = distribution.to_pytket_circuit()
render_circuit_jupyter(circ_with_dist)

Notice that ancilla qubits are added to the central module to hold the required ebits. Moreover, we see that the gates are acted on a different server from the servers where the qubits are placed. As such in this case we have one server which consists of only link qubits. Gates acting between link qubits only are referred to as 'detached' gates.

## Entanglement Distribution

When implementing starting and ending processes `pytket-dqc` will make some effort to use the minimum number of distribution operations required to implement a sequence of gates remotely, given a `Placement`. Consider for example the following three pronged star network.

In [ ]:
network = NISQNetwork([[0,1], [0,2], [0,3]], {0:[0], 1:[1], 2:[2], 3:[3]})
f = network.draw_nisq_network()

and the following simple three qubit circuit.

In [ ]:
circ = Circuit(3)
circ.add_gate(OpType.CU1, 1.0, [0, 1]).add_gate(OpType.CU1, 1.0, [0, 2])
render_circuit_jupyter(circ)

Imagine the qubits of this circuit are placed on the prongs of this network, and that the gates are placed on the modules which hold their target. Then the minimal way to distribute these gates is to create an entangled copy of the information contained in the control qubit with the central module, and then repeat from there to the two other edge modules. `pytket-dqc` will take this approach, as seen here.

In [ ]:
placement = Placement({0:1, 1:2, 2:3, 3:2, 4:3})
distribution = Distribution(HypergraphCircuit(circ), placement, network)
assert distribution.is_valid()

circ_with_dist = distribution.to_pytket_circuit()
render_circuit_jupyter(circ_with_dist)

More formally, `pytket-dqc` will consume ebits along the edges of the [Steiner tree](https://en.wikipedia.org/wiki/Steiner_tree_problem) which connects the modules containing the qubits acted on by the sequence of gates.

An additional technique we will use is embedding, which in particular is utilised by the `CoverEmbedding` `Distributor`. In some cases, embedding allows ebits to survive past Hadamard gates, allowing for their reuse. Consider the below example of the SWAP gate, which can be decomposed as a sequence of CZ gates sandwiched by Hadamard gates. Without embedding each of the 3 CZ gates would require an ebit to be distributed on the simple 2 module network given below. However, the central CZ can be embedded, and the ebit required to implement the first CZ gate remotely can be reused by the third, assuming a correction is made on the embedded portion of the circuit. Further technical details on embedding can be found in [Entanglement-efficient bipartite-distributed quantum computing with entanglement-assisted packing processes](https://arxiv.org/abs/2212.12688)

In [ ]:
from pytket_dqc.distributors import CoverEmbedding

network = NISQNetwork([[0,1]], {0:[0], 1:[1]})

circ = Circuit(2).SWAP(0,1)
DQCPass().apply(circ)
render_circuit_jupyter(circ)

distribution = CoverEmbedding().distribute(circ, network, seed=0)
circ_with_dist = distribution.to_pytket_circuit()
render_circuit_jupyter(circ_with_dist)